### 下载数据集

In [1]:
import requests

url='https://www.gutenberg.org/cache/epub/16457/pg16457.txt'
fname='./../data/AllAroundTheMoon.txt'
r = requests.get(url, stream=True, verify=False)
with open(fname, 'wb') as f:
    f.write(r.content)

d:\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [11]:
import os
import re
import collections
import time

data_dir='./../data'
token_freqs_init = collections.defaultdict(int)
with open(os.path.join(data_dir, 'AllAroundTheMoon.txt'), 'r',encoding='utf-8') as f:
    #一次性多行读取，list，每个元素是一行str
    lines = f.readlines()
    for line in lines:
        #用空格分割单词
        words=re.sub('[^A-Za-z]+', ' ', line).strip().lower().split()
        for word in words:
            #每个单词分割成字母，用空格分割开
            #在每个单词后面加</w>特殊标志
            token_freqs_init[' '.join(list(word)) + ' </w>'] += 1

print(token_freqs_init)

defaultdict(<class 'int'>, {'t h e </w>': 7549, 'p r o j e c t </w>': 97, 'g u t e n b e r g </w>': 97, 'e b o o k </w>': 13, 'o f </w>': 3644, 'a l l </w>': 475, 'a r o u n d </w>': 90, 'm o o n </w>': 477, 't h i s </w>': 480, 'i s </w>': 746, 'f o r </w>': 616, 'u s e </w>': 32, 'a n y o n e </w>': 5, 'a n y w h e r e </w>': 4, 'i n </w>': 1754, 'u n i t e d </w>': 26, 's t a t e s </w>': 30, 'a n d </w>': 1973, 'm o s t </w>': 147, 'o t h e r </w>': 169, 'p a r t s </w>': 9, 'w o r l d </w>': 59, 'a t </w>': 776, 'n o </w>': 403, 'c o s t </w>': 5, 'w i t h </w>': 700, 'a l m o s t </w>': 54, 'r e s t r i c t i o n s </w>': 2, 'w h a t s o e v e r </w>': 3, 'y o u </w>': 582, 'm a y </w>': 97, 'c o p y </w>': 12, 'i t </w>': 1157, 'g i v e </w>': 50, 'a w a y </w>': 55, 'o r </w>': 465, 'r e </w>': 63, 'u n d e r </w>': 67, 't e r m s </w>': 26, 'l i c e n s e </w>': 18, 'i n c l u d e d </w>': 3, 'o n l i n e </w>': 6, 'w w w </w>': 11, 'o r g </w>': 9, 'i f </w>': 274, 'a r e </w

In [12]:
# 初始化symbols
def update_symbols(token_freqs):
    symbols = set()
    for tokens in token_freqs.keys():
        token_list = tokens.split()
        for token in token_list:
            symbols.add(token)
    return symbols


#获取最高频率的连续字符对和freq
def get_max_freq_pair_and_freq(token_freqs):
    pairs = collections.defaultdict(int)
    for token, freq in token_freqs.items():
        symbols = token.split()
        for i in range(len(symbols) - 1):
            # “pairs”的键是两个连续符号的元组
            pairs[symbols[i], symbols[i + 1]] += freq

    max_freq_pair = max(pairs, key=pairs.get)
    pair_freq = pairs[max_freq_pair]
    return max_freq_pair,pair_freq  # 具有最大值的“pairs”键和最高频数

#获取最高频率的连续字符对
def get_max_freq_pair(token_freqs):
    pairs = collections.defaultdict(int)
    for token, freq in token_freqs.items():
        symbols = token.split()
        for i in range(len(symbols) - 1):
            # “pairs”的键是两个连续符号的元组
            pairs[symbols[i], symbols[i + 1]] += freq
    max_freq_pair = max(pairs, key=pairs.get)
    return max_freq_pair # 具有最大值的“pairs”键


def merge_symbols(max_freq_pair, token_freqs):
    new_token_freqs = dict()
    symbols = set()
    for tokens, freq in token_freqs.items():
        #合并最高频率字符对
        new_tokens = tokens.replace(' '.join(max_freq_pair),
                                  ''.join(max_freq_pair))
        #替换后的文本频率
        new_token_freqs[new_tokens] = freq

        #更新symbols
        token_list = new_tokens.split()
        for token in token_list:
            symbols.add(token)
    
    return new_token_freqs,symbols

In [13]:
#迭代
def iterative_corpus_with_freq(token_freqs, symbols, max_symbols_size):
    max_freq_pair, freq = get_max_freq_pair_and_freq(token_freqs)
    while not (len(symbols) == max_symbols_size or freq==1):
        token_freqs ,symbols = merge_symbols(max_freq_pair,token_freqs)
        max_freq_pair, freq = get_max_freq_pair_and_freq(token_freqs)
    return token_freqs,symbols

symbols_init=update_symbols(token_freqs_init)
print(symbols_init)

token_freqs,symbols = iterative_corpus_with_freq(token_freqs_init,symbols_init,10000)
print(token_freqs)
print(symbols)
print(len(symbols))


{'h', 'c', 'x', 'm', 'u', 's', 'a', 'z', 'w', 'n', 'o', 'j', 'i', 'd', 'l', 'v', 'p', 'g', 'r', '</w>', 'k', 'y', 'q', 't', 'b', 'f', 'e'}
{'the</w>': 7549, 'project</w>': 97, 'gutenberg</w>': 97, 'ebook</w>': 13, 'of</w>': 3644, 'all</w>': 475, 'around</w>': 90, 'moon</w>': 477, 'this</w>': 480, 'is</w>': 746, 'for</w>': 616, 'use</w>': 32, 'anyone</w>': 5, 'anywhere</w>': 4, 'in</w>': 1754, 'united</w>': 26, 'states</w>': 30, 'and</w>': 1973, 'most</w>': 147, 'other</w>': 169, 'parts</w>': 9, 'world</w>': 59, 'at</w>': 776, 'no</w>': 403, 'cost</w>': 5, 'with</w>': 700, 'almost</w>': 54, 'restrictions</w>': 2, 'whatsoever</w>': 3, 'you</w>': 582, 'may</w>': 97, 'copy</w>': 12, 'it</w>': 1157, 'give</w>': 50, 'away</w>': 55, 'or</w>': 465, 're</w>': 63, 'under</w>': 67, 'terms</w>': 26, 'license</w>': 18, 'included</w>': 3, 'online</w>': 6, 'www</w>': 11, 'org</w>': 9, 'if</w>': 274, 'are</w>': 280, 'not</w>': 651, 'located</w>': 7, 'will</w>': 118, 'have</w>': 436, 'to</w>': 2507, 'c

In [ ]:
#迭代
def iterative_corpus(token_freqs, symbols, max_symbols_size):
    while not len(symbols) == max_symbols_size:
        max_freq_pair= get_max_freq_pair(token_freqs)
        token_freqs ,symbols = merge_symbols(max_freq_pair,token_freqs)
    return token_freqs,symbols

symbols_init=update_symbols(token_freqs_init)
print(symbols_init)

token_freqs,symbols = iterative_corpus(token_freqs_init,symbols_init,3000)
print(token_freqs)
print(symbols)
print(len(symbols))

In [16]:
def segment_BPE(tokens, symbols):
    outputs = []
    for token in tokens:
        start, end = 0, len(token)
        cur_output = []
        # 具有符号中可能最长子字的词元段
        while start < len(token) and start < end:
            if token[start: end] in symbols:
                #匹配成功的：token[start: end]
                cur_output.append(token[start: end])
                start = end
                end = len(token)
            else:
                end -= 1
        if start < len(token):
            cur_output.append('[UNK]')
        outputs.append(' '.join(cur_output))
    return outputs

print(symbols)
tokens=['mountains</w>','ilikeeatingapples!</w>']
outputs=segment_BPE(tokens,symbols)
print(outputs)

print('like</w>' in symbols)


{'host</w>', 'contrivance</w>', 'circulation</w>', 'abashed</w>', 'lloni', 'floors</w>', 'foreground</w>', 'rely</w>', 'brit', 'provided</w>', 'express</w>', 'responsibility</w>', 'sail</w>', 'less</w>', 'trail</w>', 'lighted</w>', 'lim', 'raged</w>', 'descending</w>', 'ob', 'wickedest</w>', 'instantaneously</w>', 'pre', 'tolerably</w>', 'mighty</w>', 'upside</w>', 'ages</w>', 'lexico', 'ample</w>', 'momen', 'contrast</w>', 'spend</w>', 'glow</w>', 'appeal</w>', 'statu', 'periselene</w>', 'consists</w>', 'guard</w>', 'nay</w>', 'expressing</w>', 'anchored</w>', 'localities</w>', 'year', 'matters</w>', 'dabs</w>', 'mille</w>', 'honorable</w>', 'ominously</w>', 'visedly</w>', 'suffici', 'decisive</w>', 'safe</w>', 'wishing</w>', 'glancing</w>', 'uphea', 'prolongation</w>', 'past</w>', 'laid</w>', 'handed</w>', 'appreciable</w>', 'reserved</w>', 'villa', 'liability</w>', 'immediately</w>', 'pack</w>', 'naples</w>', 'anxiety</w>', 'discovering</w>', 'obeyed</w>', 'diameter</w>', 'stooped</